In [1]:
import os
import torch
from sklearn import metrics
import logging
from timeit import default_timer as timer
import pandas as pn

from context import resnet
from context import utils
from context import data_loader
from context import data_utils

STORE_RESULT_PATH = os.getcwd()
BINARY_PATH = 'binary-classification/'

In [9]:
#######################################################################
# CODE TO CONVERT EXPERIMENT RESULTS OF RESNET50 INTO DEDICATED
# CLASSIFICATION REPORT-CSV FILES.
# FOR MULTI-CLASS CLASSIFICATION AND BINARY CLASSIFICATION.
# For ResNet50
#######################################################################

# load results without any skipped connections
df = pn.read_csv(os.path.join(STORE_RESULT_PATH, 'resnet50_experiment_results.csv'))

def convertStringToBoolArray(string: str):
    if type(string) == list:
        return string
    temp = [item.replace('[', '').replace(']', '').strip() for item in string.split(' ')]
    return [(x == 'True') for x in temp if len(x) == len('True') or len(x) == len('False')]

def convertStringToStringArray(string: str):
    return [
            x.replace('[', '')
            .replace(']', '')
            .replace("'", '')
            .strip() 
            for x in string.split(', ')
        ]
def filterForBinaryClassification(classes_list):
    plant_classes = [
        'Tulipa gesneriana', 'angiosperm', 
        'aster', 'floret', 'flower cluster', 
        'gazania', 'grape', 'hepatica', 
        'Night-blooming cereus', 'prairie gentian', 
        'raspberry', 'wild calla']
    temp_cls = []
    for name in classes_list:
        temp_cls.append('plant' if name in plant_classes else 'no_plant')
    return temp_cls

def getTrueIndexList(bool_list):
    idx_ls = []
    for idx, val in enumerate(bool_list):
        if val:
            idx_ls.append(idx)
    return idx_ls

def generateClassificationReports(df):
    for n in df.skip_n.unique():
        entries = df[df['skip_n'] == n]
        max_acc = entries['acc'].max()

        max_entry = entries[entries['acc'] == max_acc]

        max_entry = max_entry.iloc[0]
        
        grndT = max_entry.ground_truth
        pred = max_entry.prediction

        report_dict = metrics.classification_report(grndT, pred, digits=3, output_dict=True)
        logging.info(metrics.classification_report(grndT, pred, digits=3))
        
        temp_df = pn.DataFrame(report_dict).transpose()
        file_path = os.path.join(os.getcwd(), f'{n}-resnet50-classification-report.csv')
        temp_df.to_csv(file_path, index=False)

def generateClassificationReportsBinary(df):
    for n in df.skip_n.unique():
        entries = df[df['skip_n'] == n]
        max_acc = entries['acc'].max()
        max_entry = entries[entries['acc'] == max_acc]
        max_entry = max_entry.iloc[0]
        
        grndT = max_entry.ground_truth
        pred = max_entry.prediction

        report_dict = metrics.classification_report(grndT, pred, digits=3,output_dict=True)
        logging.info(metrics.classification_report(grndT, pred, digits=3))
        
        temp_df = pn.DataFrame(report_dict).transpose()
        file_path = os.path.join(BINARY_PATH, f'{n}-resnet50-bin-classification-report.csv')
        if not os.path.exists(BINARY_PATH):
            os.mkdir(BINARY_PATH)
        temp_df.to_csv(file_path, index=False)

# preprocess the csv-strings
df['configuration'] = df['configuration'].apply(lambda item: convertStringToBoolArray(item))
df['ground_truth'] = df['ground_truth'].apply(lambda item: convertStringToStringArray(item))
df['prediction'] = df['prediction'].apply(lambda item: convertStringToStringArray(item))

generateClassificationReports(df)

# convert to binary classification
df['ground_truth'] = df['ground_truth'].apply(lambda item: filterForBinaryClassification(item))
df['prediction'] = df['prediction'].apply(lambda item: filterForBinaryClassification(item))
generateClassificationReportsBinary(df)

In [19]:
from collections import Counter

#######################################################################
# CODE TO CONVERT EXPERIMENT DATA INTO DEDICATED
# MULTI-CLASS AND BINARY CLASSIFICATIION REPORTS.
# For MSDNet5
#######################################################################

# load experiment data from csv file
df = pn.read_csv(os.path.join('../reports', 'report-msdnet5-run.csv'))

def convertStringToBoolArray(string: str):
    if type(string) == list:
        return string
    temp = [item.replace('[', '').replace(']', '').strip() for item in string.split(' ')]
    return [(x == 'True') for x in temp if len(x) == len('True') or len(x) == len('False')]

def convertStringToStringArray(string: str):
    return [
            x.replace('[', '')
            .replace(']', '')
            .replace("'", '')
            .strip() 
            for x in string.split(', ')
        ]

def filterForBinaryClassification(classes_list):
    plant_classes = [
        'Tulipa gesneriana', 'angiosperm', 
        'aster', 'floret', 'flower cluster', 
        'gazania', 'grape', 'hepatica', 
        'Night-blooming cereus', 'prairie gentian', 
        'raspberry', 'wild calla']
    temp_cls = []
    for name in classes_list:
        temp_cls.append('plant' if name in plant_classes else 'no_plant')
    return temp_cls

df['ground_truth'] = df['ground_truth'].apply(lambda item: convertStringToStringArray(item))
df['prediction'] = df['prediction'].apply(lambda item: convertStringToStringArray(item))


def generateClassificationReports(df):
    for classifier in df.classifier.unique():
        entry = df[df['classifier'] == classifier]
        entry = entry.iloc[0]

        # have been switched due to wrong
        # generation in report.
        pred = entry.ground_truth#.values[0]
        grndT = entry.prediction#.values[0]

        report_dict = metrics.classification_report(grndT, pred, digits=3, output_dict=True)
        logging.info(metrics.classification_report(grndT, pred, digits=3))

        temp_df = pn.DataFrame(report_dict).transpose()
        file_path = os.path.join(os.getcwd(), f'{classifier}-msdnet5-classification-report.csv')
        temp_df.to_csv(file_path, index=False) 

def generateClassificationReportsBinary(df):
    for classifier in df.classifier.unique():
        entry = df[df['classifier'] == classifier]
        entry = entry.iloc[0]
        
        # have been switched due to wrong
        # generation in report.
        pred = entry.ground_truth#.values[0]
        grndT = entry.prediction#.values[0]

        report_dict = metrics.classification_report(grndT, pred, digits=3, output_dict=True)
        logging.info(metrics.classification_report(grndT, pred, digits=3))

        temp_df = pn.DataFrame(report_dict).transpose()
        file_path = os.path.join(BINARY_PATH, f'{classifier}-bin-msdnet5-classification-report.csv')
        temp_df.to_csv(file_path, index=False) 


generateClassificationReports(df)

df['ground_truth'] = df['ground_truth'].apply(lambda item: filterForBinaryClassification(item))
df['prediction'] = df['prediction'].apply(lambda item: filterForBinaryClassification(item))
generateClassificationReportsBinary(df)

In [20]:
#######################################################################
# CODE TO CONVERT EXPERIMENT DATA INTO DEDICATED
# MULTI-CLASS AND BINARY CLASSIFICATIION REPORTS.
# For DenseNet121
#######################################################################

def convertStringToBoolArray(string: str):
    if type(string) == list:
        return string
    temp = [item.replace('[', '').replace(']', '').strip() for item in string.split(' ')]
    return [(x == 'True') for x in temp if len(x) == len('True') or len(x) == len('False')]

def convertStringToStringArray(string: str):
    return [
            x.replace('[', '')
            .replace(']', '')
            .replace("'", '')
            .strip() 
            for x in string.split(', ')
        ]

def filterForBinaryClassification(classes_list):
    plant_classes = [
        'Tulipa gesneriana', 'angiosperm', 
        'aster', 'floret', 'flower cluster', 
        'gazania', 'grape', 'hepatica', 
        'Night-blooming cereus', 'prairie gentian', 
        'raspberry', 'wild calla']
    temp_cls = []
    for name in classes_list:
        temp_cls.append('plant' if name in plant_classes else 'no_plant')
    return temp_cls

def generateClassificationReports(df):
    for n in df.skip_n.unique():
        entries = df[df['skip_n'] == n]
        max_acc = entries['acc'].max()

        max_entry = entries[entries['acc'] == max_acc]

        max_entry = max_entry.iloc[0]
        
        grndT = max_entry.ground_truth
        pred = max_entry.prediction

        report_dict = metrics.classification_report(grndT, pred, digits=3, output_dict=True)
        logging.info(metrics.classification_report(grndT, pred, digits=3))
        
        temp_df = pn.DataFrame(report_dict).transpose()
        file_path = os.path.join(os.getcwd(), f'{n}-densenet121-classification-report.csv')
        temp_df.to_csv(file_path, index=False)

def generateClassificationReportsBinary(df):
    for n in df.skip_n.unique():
        entries = df[df['skip_n'] == n]
        max_acc = entries['acc'].max()
        max_entry = entries[entries['acc'] == max_acc]
        max_entry = max_entry.iloc[0]
        
        grndT = max_entry.ground_truth
        pred = max_entry.prediction

        report_dict = metrics.classification_report(grndT, pred, digits=3,output_dict=True)
        logging.info(metrics.classification_report(grndT, pred, digits=3))
        
        temp_df = pn.DataFrame(report_dict).transpose()
        file_path = os.path.join(BINARY_PATH, f'{n}-densenet121-bin-classification-report.csv')
        if not os.path.exists(BINARY_PATH):
            os.mkdir(BINARY_PATH)
        temp_df.to_csv(file_path, index=False)

# load from source report
df = pn.read_csv(os.path.join('../reports', 'report-densenet121-run.csv'))

# convert strings into array
df['ground_truth'] = df['ground_truth'].apply(lambda item: convertStringToStringArray(item))
df['prediction'] = df['prediction'].apply(lambda item: convertStringToStringArray(item))

generateClassificationReports(df)

# adapt to binary classification
df['ground_truth'] = df['ground_truth'].apply(lambda item: filterForBinaryClassification(item))
df['prediction'] = df['prediction'].apply(lambda item: filterForBinaryClassification(item))

generateClassificationReportsBinary(df)